<a href="https://colab.research.google.com/github/santyxd3000/Undergraduate-Project/blob/main/Classifying_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROYECTO DE GRADO - MODELOS DE CLASIFICACIÓN**

Santiago Pardo Morales - 202013025

In [5]:
#For using the GPU
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
***********************************************************************
We will now install RAPIDS via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached cupy_cuda12x-12.3.0-cp310-cp310-manylinux2014_x86_64.whl (81.8 MB)

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update y

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import datetime
import scipy
sns.set()
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import io
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import cudf
import cuml
import cupy as cp
from cuml import preprocessing as cpp
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
physical_devices = tf.config.list_physical_devices('GPU')
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import NearMiss
from cuml.svm import SVC
from cuml.multiclass import OneVsOneClassifier
from cuml.multiclass import OneVsRestClassifier

ValueError: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 88 from C header, got 72 from PyObject

In [ ]:
from collections import Counter

In [ ]:
print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

In [ ]:
#Leer datos etiquetados
df = pd.read_csv('df_with_labels.csv',sep=';').dropna()

In [ ]:
#Counting each item
print(sorted(Counter(df['Label']).items()))

In [ ]:
#Time series
train_data = df[df['Timestamp']<='2021-09-03']
print(sorted(Counter(train_data['Label']).items()))

In [ ]:
val_test_data = df[df['Timestamp']>='2021-09-03']

In [ ]:
#Time series
val_data = val_test_data[val_test_data['Timestamp']<='2022-05-16']
print(sorted(Counter(val_data['Label']).items()))

In [ ]:
#Time series
test_data = val_test_data[val_test_data['Timestamp']>='2022-05-16']
print(sorted(Counter(test_data['Label']).items()))

## **MÉTODO 1 - RED NEURONAL**

In [ ]:
df['Label'].replace({'No Fault': 0, 'Shading': 1, 'Open Circuit': 2, 'Short Circuit': 3, 'Unknown':4}, inplace=True)

In [ ]:
df[df['Label']==1].shape

In [ ]:
x_data = df[['U_DC1','U_DC2','I_DC1','I_DC2']]
y_data = df[['Label']]

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_data)
y_data = np.array(y_data)

In [ ]:
#Divide data
x_train, x_temp, y_train, y_temp = train_test_split(x_scaled, y_data, train_size = 0.6, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size = 0.5, random_state=42)

### Red Neuronal - Solo capa de entrada y de salida

In [ ]:
ann_1 = Sequential()
ann_1.add(Dense(4, input_shape=(x_train.shape[1],),activation='relu'))

#output layer with softmax function
ann_1.add(Dense(5,activation='softmax'))

#compile model
ann_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_1.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_1.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

### Red Neuronal - Capa intermedia entre entrada y salida

In [ ]:
ann_2 = Sequential()
ann_2.add(Dense(16, input_shape=(x_train.shape[1],),activation='relu'))
ann_2.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_2.add(Dense(5,activation='softmax'))

#compile model
ann_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_2.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_2.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

### Red Neuronal - 2 capas intermedia entre entrada y salida

In [ ]:
ann_3 = Sequential()
ann_3.add(Dense(64, input_shape=(x_train.shape[1],),activation='relu'))
ann_3.add(Dense(16 ,activation='relu'))
ann_3.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_3.add(Dense(5,activation='softmax'))

#compile model
ann_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_3.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_3.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

### Red Neuronal - 3 capas intermedia entre entrada y salida

In [ ]:
ann_4 = Sequential()
ann_4.add(Dense(256, input_shape=(x_train.shape[1],),activation='relu'))
ann_4.add(Dense(64 ,activation='relu'))
ann_4.add(Dense(16 ,activation='relu'))
ann_4.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_4.add(Dense(5,activation='softmax'))

#compile model
ann_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_4.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_4.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

### Red Neuronal - 4 capas intermedia entre entrada y salida

In [ ]:
ann_5 = Sequential()
ann_5.add(Dense(256, input_shape=(x_train.shape[1],),activation='relu'))
ann_5.add(Dense(64 ,activation='relu'))
ann_5.add(Dense(16 ,activation='relu'))
ann_5.add(Dense(4 ,activation='relu'))
ann_5.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_5.add(Dense(5,activation='softmax'))

#compile model
ann_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_5.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_5.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

### Modelo Escogido - Red Neuronal con 2 capas intermedias entre entrada y salida

In [ ]:
ann_predict_test = ann_3.predict(x_test)

In [ ]:
ann_predict_test = np.argmax(ann_predict_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

labels = ['NF','S','OC','SC','U']

cm = confusion_matrix(y_test, ann_predict_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

plt.figure(figsize=(30,8))
disp.plot()
plt.title("Confusion Matrix - ANN - Test Data")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, ann_predict_test)
precision = precision_score(y_test, ann_predict_test,average='weighted')
recall = recall_score(y_test, ann_predict_test,average='weighted')
f1_score = f1_score(y_test, ann_predict_test,average='weighted')

In [ ]:
print("Accuracy: "+ str(accuracy))
print("Precision: "+ str(precision))
print("Recall: "+ str(recall))
print("F1-Score: "+ str(f1_score))

## **MÉTODO 2 - SUPPORT VECTOR MACHINES**

In [ ]:
cu_df = cudf.DataFrame(df)

In [ ]:
x_data = cu_df[['U_DC1','U_DC2','I_DC1','I_DC2']]
y_data = cu_df[['Label']]

In [ ]:
cu_scaler = cuml.preprocessing.StandardScaler()
x_scaled = cu_scaler.fit_transform(x_data)
y_data = y_data.to_cupy()

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(x_scaled, y_data, train_size = 0.6, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size = 0.5, random_state=42)

x_train = x_train.to_numpy()
y_train = y_train.ravel()
y_train = cp.asnumpy(y_train)
x_val = x_val.to_numpy()
y_val = y_val.ravel()
y_val = cp.asnumpy(y_val)
x_test = x_test.to_numpy()
y_test = y_test.ravel()
y_test = cp.asnumpy(y_test)

In [ ]:
svc_classifier = OneVsRestClassifier(SVC(kernel='rbf',gamma='scale',random_state=42))

param_grid = {
    "estimator__C": np.array([0.001,0.01,0.1,1,10,100,1000]),
}

grid_search_ovr = cuml.model_selection.GridSearchCV(svc_classifier, param_grid, scoring='accuracy', cv=5, refit='accuracy',n_jobs=-1)

grid_search_ovr.fit(x_train, y_train)

print("Best Parameters for SVM with One vs Rest Classification and RBF Kernel with Gamma Scale: ", grid_search_ovr.best_params_)

In [ ]:
svc_classifier = OneVsRestClassifier(SVC(kernel='rbf',gamma='auto'))

param_grid = {
    "estimator__C": np.array([0.001,0.01,0.1,1,10,100,1000,10000]),
}

grid_search_ovr = cuml.model_selection.GridSearchCV(svc_classifier, param_grid, scoring='accuracy', cv=5, refit='accuracy',n_jobs=-1)

grid_search_ovr.fit(x_train, y_train)

print("Best Parameters for SVM with One vs Rest Classification RBF Kernel and Gamma Auto: ", grid_search_ovr.best_params_)

In [ ]:
svc_classifier = OneVsRestClassifier(SVC(kernel='sigmoid',random_state=42))

param_grid = {
    "estimator__C": np.array([0.001,0.01,0.1,1,10,100,1000,10000]),
}

grid_search_ovr = cuml.model_selection.GridSearchCV(svc_classifier, param_grid, scoring='accuracy', cv=5, refit='accuracy',n_jobs=-1)

grid_search_ovr.fit(x_train, y_train)

print("Best Parameters for SVM with One vs Rest Classification and Sigmoid Kernel: ", grid_search_ovr.best_params_)

In [ ]:
svc_classifier = OneVsOneClassifier(SVC(kernel='rbf',gamma='scale',random_state=42))

param_grid = {
    "estimator__C": np.array([0.001,0.01,0.1,1,10,100,1000]),
}

grid_search_ovr = cuml.model_selection.GridSearchCV(svc_classifier, param_grid, scoring='accuracy', cv=5, refit='accuracy',n_jobs=-1)

grid_search_ovr.fit(x_train, y_train)

print("Best Parameters for SVM with One vs One Classification and RBF Kernel with Gamma Scale: ", grid_search_ovr.best_params_)

In [ ]:
svc_classifier = OneVsOneClassifier(SVC(kernel='rbf',gamma='auto',random_state=42))

param_grid = {
    "estimator__C": np.array([0.001,0.01,0.1,1,10,100,1000,10000]),
}

grid_search_ovr = cuml.model_selection.GridSearchCV(svc_classifier, param_grid, scoring='accuracy', cv=5, refit='accuracy',n_jobs=-1)

grid_search_ovr.fit(x_train, y_train)

print("Best Parameters for SVM with One vs One Classification RBF Kernel and Gamma Auto: ", grid_search_ovr.best_params_)

In [ ]:
svc_classifier = OneVsOneClassifier(SVC(kernel='sigmoid',random_state=42))

param_grid = {
    "estimator__C": np.array([0.001,0.01,0.1,1,10,100,1000,10000]),
}

grid_search_ovr = GridSearchCV(svc_classifier, param_grid, scoring='accuracy', cv=5, refit='accuracy',n_jobs=-1)

grid_search_ovr.fit(x_train, y_train)

print("Best Parameters for SVM with One vs One Classification and Sigmoid Kernel: ", grid_search_ovr.best_params_)